In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


df1 = pd.read_csv("bundesliga.csv")
df2 = pd.read_csv("bundesliga2.csv")

df = pd.concat([df1, df2], ignore_index=True)

if 'DATE' in df.columns:
    df['DATE'] = pd.to_datetime(df['DATE'], errors='coerce')


home = df[['SEASON', 'HOMETEAM', 'FTHG', 'FTAG']].copy()
home['Team'] = home['HOMETEAM']
home['Points'] = np.where(home['FTHG'] > home['FTAG'], 3,
                          np.where(home['FTHG'] == home['FTAG'], 1, 0))
home['GF'] = home['FTHG']
home['GA'] = home['FTAG']
home = home[['SEASON', 'Team', 'Points', 'GF', 'GA']]

away = df[['SEASON', 'AWAYTEAM', 'FTHG', 'FTAG']].copy()
away['Team'] = away['AWAYTEAM']
away['Points'] = np.where(away['FTAG'] > away['FTHG'], 3,
                          np.where(away['FTAG'] == away['FTHG'], 1, 0))
away['GF'] = away['FTAG']
away['GA'] = away['FTHG']
away = away[['SEASON', 'Team', 'Points', 'GF', 'GA']]

combined = pd.concat([home, away], ignore_index=True)

season_table = combined.groupby(['SEASON', 'Team'], as_index=False).agg({
    'Points': 'sum',
    'GF': 'sum',
    'GA': 'sum'
})
season_table['GD'] = season_table['GF'] - season_table['GA']


season_table = season_table.sort_values(['SEASON', 'Points', 'GD', 'GF'], 
                                          ascending=[True, False, False, False])
season_table['FinalRank'] = season_table.groupby('SEASON').cumcount() + 1

print(season_table.head())


all_seasons = season_table['SEASON'].unique()
season_colors = {}

for s in all_seasons:
    season_data = season_table[season_table['SEASON'] == s]
    teams_in_season = sorted(season_data['Team'].unique())
    n_teams = len(teams_in_season)
    
    palette = sns.color_palette('Set3', n_teams)
    
    color_dict = {team: palette[i] for i, team in enumerate(teams_in_season)}
    season_colors[s] = color_dict

output_folder = "Season_GD_Bars"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
for s in all_seasons:
    season_data = season_table[season_table['SEASON'] == s].copy()
    season_data = season_data.sort_values(by='GD', ascending=False)
    
    color_map = season_colors[s]
    bar_colors = season_data['Team'].map(color_map)
    
    plt.figure(figsize=(8, 6))
    plt.barh(season_data['Team'], season_data['GD'], color=bar_colors)
    plt.gca().invert_yaxis()  
    plt.title(f"Season: {s} - Goal Difference by Team")
    plt.xlabel("Goal Difference (GD)")
    plt.ylabel("Team")
    
    for index, row in season_data.iterrows():
        plt.text(row['GD'] + 0.5, list(season_data['Team']).index(row['Team']), 
                 str(row['GD']), va='center', fontsize=9)
    
    plt.tight_layout()
    plt.savefig(os.path.join(output_folder, f"{s}_GD_Bar.png"))
    plt.close()

print("Plots have been saved in the folder:", output_folder)


    SEASON            Team  Points    GF    GA    GD  FinalRank
0     1994   Bayern Munich      61  68.0  37.0  31.0          1
8     1994  Kaiserslautern      61  64.0  36.0  28.0          2
1     1994        Dortmund      54  49.0  45.0   4.0          3
4     1994   Ein Frankfurt      53  57.0  41.0  16.0          4
11    1994      Leverkusen      53  60.0  47.0  13.0          5
Plots have been saved in the folder: Season_GD_Bars
